In [143]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime

import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
%matplotlib inline

# 열 내 데이터가 생략되지 않도록 설정
pd.set_option('display.max_colwidth', None)

In [144]:
# 불필요한 경고문 생략(선택)
import warnings
warnings.filterwarnings('ignore')

# 모든 컬럼 출력설정(선택)
pd.set_option('display.max_columns', None)

# 데이터 로드
df = pd.read_csv('onlinesales_merge.csv')

plt.rc('font', family='NanumGothic') # For Windows
%matplotlib inline

In [145]:
df.head(10)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,쿠폰코드,할인율
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,ELEC10,10.0
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,ELEC10,10.0
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,남,Chicago,12,OFF10,10.0
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,남,Chicago,12,SALE10,10.0
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,남,Chicago,12,AIO10,10.0
5,USER_1358,Transaction_0003,2019-01-01,Product_0854,Bags,15,5.15,6.5,Used,남,Chicago,12,AIO10,10.0
6,USER_1358,Transaction_0003,2019-01-01,Product_0880,Drinkware,15,3.08,6.5,Not Used,남,Chicago,12,EXTRA10,10.0
7,USER_1358,Transaction_0003,2019-01-01,Product_0885,Drinkware,15,10.31,6.5,Clicked,남,Chicago,12,EXTRA10,10.0
8,USER_1358,Transaction_0003,2019-01-01,Product_0898,Drinkware,5,9.27,6.5,Used,남,Chicago,12,EXTRA10,10.0
9,USER_1358,Transaction_0013,2019-01-01,Product_0973,Lifestyle,1,1.24,6.5,Used,남,Chicago,12,EXTRA10,10.0


# 매출 관련 지표 확인

In [146]:
#평균 금액 정보 확인 
## 평균금액은 할인가 미 포함,최초 금액 
df.groupby(['제품ID','수량','배송료','평균금액'])['할인율'].nunique().reset_index().sort_values(by='할인율',ascending=False)

,제품ID,수량,배송료,평균금액,할인율
1319,Product_0184,2,6.50,1.59,3
5162,Product_0506,1,6.00,11.75,3
12708,Product_0946,1,6.50,4.40,3
1572,Product_0195,1,6.50,3.99,3
5129,Product_0498,1,6.00,18.99,3
...,...,...,...,...,...
6337,Product_0649,2,6.00,19.00,1
6338,Product_0649,2,6.00,23.75,1
6340,Product_0650,1,6.00,28.50,1
6342,Product_0650,1,12.99,22.80,1


In [147]:
#평균 금액 정보 확인 
#쿠폰상태와 할인율에 상관없이 가격이 일정함 => 적용이 안된것으로 보임 
df.groupby(['고객ID','거래날짜','제품ID','수량','배송료','평균금액'])['쿠폰상태'].nunique().reset_index().sort_values(by='쿠폰상태',ascending=False)

,고객ID,거래날짜,제품ID,수량,배송료,평균금액,쿠폰상태
45696,USER_1406,2019-01-16,Product_0981,1,6.5,153.71,3
11452,USER_0353,2019-01-14,Product_0981,1,6.5,153.71,3
20519,USER_0643,2019-02-06,Product_0976,1,6.5,119.00,3
14007,USER_0443,2019-02-23,Product_0981,1,6.5,149.00,3
38674,USER_1218,2019-12-15,Product_0990,1,6.5,202.84,3
...,...,...,...,...,...,...,...
16251,USER_0535,2019-11-26,Product_0994,1,6.5,349.00,1
16252,USER_0535,2019-11-26,Product_0995,3,6.5,279.00,1
16253,USER_0535,2019-11-27,Product_0183,1,6.5,2.39,1
16254,USER_0535,2019-11-27,Product_0195,1,6.0,3.99,1


# 매출관련 파생 변수 생성


In [148]:
#총 주문 금액(매출) 정의 : 수량 *평균 금액 *(1-쿠폰 적용할인율)

#할인금액 : 평균금액 * 할인율 (개별 제품별 할인율 적용 )
# df['총 할인금액'] = df.apply(lambda x:x['평균금액']*(df['할인율']*0.01)*x['수량'] if x['쿠폰상태']=='Used' else 0,axis=1)

#총 할인금액 컬럼 생성 : 위에 람다식 최적화 코드
mask = (df['쿠폰상태'] == 'Used')
df.loc[mask, '할인금액'] = df.loc[mask, '평균금액'] * (df.loc[mask, '할인율'] * 0.01) * df.loc[mask, '수량']
df.loc[~mask, '할인금액'] = 0  # 나머지는 0으로 처리

#총 매출 / 수익 컬럼 생성
df['매출'] = (df['수량']*df['평균금액'])
df['수익'] = (df['수량']*df['평균금액'])- df['할인금액']
df


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,쿠폰코드,할인율,할인금액,매출,수익
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,남,Chicago,12,OFF10,10.0,0.205,2.05,1.845
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,남,Chicago,12,SALE10,10.0,0.000,87.65,87.650
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,남,Chicago,12,AIO10,10.0,1.650,16.50,14.850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,ELEC30,30.0,0.000,121.30,121.300
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,SALE30,30.0,14.676,48.92,34.244
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,ELEC30,30.0,45.564,151.88,106.316
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,ELEC30,30.0,0.000,402.60,402.600


In [149]:
#주문내역별 배송료 동일한지 여부 확인 -> o 
df.groupby(['고객ID','거래ID','거래날짜'])['배송료'].nunique().sort_values(ascending=False)

고객ID       거래ID               거래날짜      
USER_0000  Transaction_16900  2019-09-15    1
USER_0916  Transaction_6543   2019-04-14    1
USER_0996  Transaction_24866  2019-12-28    1
           Transaction_24865  2019-12-28    1
           Transaction_0758   2019-01-11    1
                                           ..
USER_0532  Transaction_24312  2019-12-19    1
           Transaction_24311  2019-12-19    1
           Transaction_24310  2019-12-19    1
           Transaction_24309  2019-12-19    1
USER_1467  Transaction_18638  2019-10-10    1
Name: 배송료, Length: 26631, dtype: int64

In [150]:
# 고객 주문 단위 집계 : sales_per_order df 생성 -> 총 매출 : 수량*평균금액 +주문 건당 배송비
sales_per_order = df.groupby(['고객ID','거래ID','거래날짜']).agg({'배송료':'max','매출':'sum'}).reset_index()
sales_per_order['총 매출(배송비포함)'] = sales_per_order['매출'] +  sales_per_order['배송료']

df = df.merge(sales_per_order[['고객ID','거래ID','거래날짜','총 매출(배송비포함)']],on=['고객ID', '거래ID', '거래날짜'],how='left')

In [151]:
#고객의 주문건수별 총 매출 동일한지 확인  
df.groupby(['고객ID','거래ID','거래날짜'])['총 매출(배송비포함)'].nunique().sort_values(ascending=False)

고객ID       거래ID               거래날짜      
USER_0000  Transaction_16900  2019-09-15    1
USER_0916  Transaction_6543   2019-04-14    1
USER_0996  Transaction_24866  2019-12-28    1
           Transaction_24865  2019-12-28    1
           Transaction_0758   2019-01-11    1
                                           ..
USER_0532  Transaction_24312  2019-12-19    1
           Transaction_24311  2019-12-19    1
           Transaction_24310  2019-12-19    1
           Transaction_24309  2019-12-19    1
USER_1467  Transaction_18638  2019-10-10    1
Name: 총 매출(배송비포함), Length: 26631, dtype: int64

In [ ]:
df

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,쿠폰코드,할인율,할인금액,매출,수익,총 매출(배송비포함)
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339,160.21
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339,160.21
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,남,Chicago,12,OFF10,10.0,0.205,2.05,1.845,8.55
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,남,Chicago,12,SALE10,10.0,0.000,87.65,87.650,435.10
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,남,Chicago,12,AIO10,10.0,1.650,16.50,14.850,435.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,ELEC30,30.0,0.000,121.30,121.300,127.80
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,SALE30,30.0,14.676,48.92,34.244,55.42
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,ELEC30,30.0,45.564,151.88,106.316,158.38
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,ELEC30,30.0,0.000,402.60,402.600,409.10


In [ ]:
# 최종 전처리 테이블 저장 
# df.to_csv("onlinesales_final.csv",index=False)

In [154]:
data=pd.read_csv("onlinesales_final.csv")

In [155]:
data

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,쿠폰코드,할인율,할인금액,매출,수익,총 매출(배송비포함)
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339,160.21
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,ELEC10,10.0,15.371,153.71,138.339,160.21
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,남,Chicago,12,OFF10,10.0,0.205,2.05,1.845,8.55
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,남,Chicago,12,SALE10,10.0,0.000,87.65,87.650,435.10
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,남,Chicago,12,AIO10,10.0,1.650,16.50,14.850,435.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,ELEC30,30.0,0.000,121.30,121.300,127.80
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,SALE30,30.0,14.676,48.92,34.244,55.42
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,ELEC30,30.0,45.564,151.88,106.316,158.38
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,ELEC30,30.0,0.000,402.60,402.600,409.10
